In [1]:
# This script demonstrates the capabilities of cluster_slug by using
# it to re-analyze the M31 star clusters catalog of Johnson et al. (2015, ApJ)

# Import the libraries we'll need
from astropy.io import ascii
from astropy import table
from astropy.table import unique, Table, hstack, Column
import sys
from numpy import inf
import numpy as np
import numpy.ma as ma
import time
from matplotlib.pyplot import figure as fig
import matplotlib.pyplot as plt
from scipy import stats
from scipy.stats import norm
from slugpy.cluster_slug import cluster_slug
from slugpy import * 
import matplotlib.pyplot as plt
from matplotlib.lines import Line2D
import matplotlib.lines as mlines
from matplotlib.ticker import (MultipleLocator, FormatStrFormatter,
                               AutoMinorLocator)

In [2]:
#Imported AP (M33) fits files which include best fit of mass and age data already

m33phot_test=Table.read('/Users/user/Dropbox/m33_zoo_data/data_v2/apm33_6phot_test.fits')

LEGUS_PHAT_PadKroupa_prop=Table.read('LEGUS_1e6_1e10age_PadKroupaAfter_PHAT_lib_1e7Trials_cluster_prop.fits')
LEGUS_PHAT_PadKroupa_phot=Table.read('LEGUS_1e6_1e10age_PadKroupaAfter_PHAT_lib_1e7Trials_cluster_phot.fits')

dmod_M33=24.65 #De Grijs et al 2014 (AJ 148:17)

In [3]:
m33phot_test.remove_columns(['MAG275','MAG110','MAG160','SIG275','SIG110','SIG160','DETSIG275','DETSIG110','DETSIG160'])

In [4]:
ind336DS3=np.where(m33phot_test['DETSIG336']<3.0)
m33phot_test.remove_rows(ind336DS3)
m33phot_test

ID,X,Y,PIXRAD,FX,FY,RA,DEC,NUMOBJ,VIEWFRAC,CFRAC,OFRAC,GFRAC,EFRAC,CCNT,CPCNT,GCNT,GPCNT,ECNT,EPCNT,CLASSIFICATION_COUNT,PRIMARY_APCAN,SUBIMG,BRICK,FIELD,IMG,TOOTH,SUBJECT_ID,NUMOBJALL,NOSOLFLAG,EDGES,VIEWFRACS,EXCLUDEFLAGS,PIXHLR475,MAG336,SIG336,DETSIG336,MAG475,SIG475,DETSIG475,MAG814,SIG814,DETSIG814
int32,float32,float32,float32,float32,float32,float64,float64,int32,float32,float32,float32,float32,float32,int32,int32,int32,int32,int32,int32,int16,int32,bytes17,int16,int16,int16,int16,int32,int32,int16,bytes36,bytes49,bytes14,float32,float32,float32,float32,float32,float32,float32,float32,float32,float32
3,511.167,419.4067,30.805826,2386.167,2019.4067,23.60606421401875,30.699199021851136,1,1.0,1.0,0.0,0.0,0.0,60,0,0,0,0,0,60,27021,M33-B02-F01-TE_20,2,1,20,1,28526173,4,0,"80.6,4.5,6.1,20.3,","1.0000,0.2833,0.0833,0.8361,","0,1,1,1,",8.728445,20.486721,0.11886143,5.8726273,20.217335,0.1563142,6.99401,19.335989,0.27782437,3.9929368
4,274.12375,430.10718,38.054737,1524.1238,430.10718,23.43433303773043,30.51408232617703,1,1.0,0.98333335,0.016666668,0.0,0.016666668,59,0,0,0,1,0,60,21254,M33-B03-F16_03,3,16,3,-1,28525713,2,0,"69.9,28.9,","1.0000,0.9000,","0,1,",13.384977,17.207281,0.07225124,9.632259,19.698397,0.08457241,12.864533,18.16659,0.15653984,6.9838634
5,224.73451,423.41208,37.18926,849.7345,1623.4121,23.583845918691235,30.659212182808222,1,0.9836066,0.9166667,0.083333336,0.0,0.083333336,55,0,0,0,5,0,61,9815,M33-B02-F07_14,2,7,14,-1,28524968,2,0,"76.6,23.6,","0.9836,0.8548,","0,1,",9.4648075,18.9727,0.11130678,6.2656794,18.70199,0.023569176,46.085743,17.94467,0.06835622,15.904741
7,248.0658,140.99915,39.348427,873.0658,140.99915,23.390373394289195,30.6674512161078,2,0.96694213,0.974359,0.025641026,0.0,0.025641026,114,0,0,0,3,0,121,11623,M33-B02-F12_02,2,12,2,-1,28525096,2,0,"83.6,141.0,","0.9833,0.9508,","0,0,",12.779946,19.286543,0.14845498,4.710332,18.826597,0.08265972,13.160799,17.740986,0.16589268,6.5956807
8,203.50774,440.90045,63.505905,1453.5077,2440.9004,23.479685106627514,30.799434542708635,1,0.98333335,0.9830508,0.016949153,0.0,0.016949153,58,0,0,0,1,0,60,4403,M33-B01-F11_23,1,11,23,-1,28524585,2,1,"41.7,59.1,","0.9833,0.9833,","0,1,",16.818165,16.588139,0.054756697,12.701803,16.79031,0.03153818,34.436665,16.108368,0.0520216,20.886917
11,119.026726,403.0801,33.970577,744.02673,2403.08,23.445514019227883,30.7663251695561,1,0.98333335,1.0,0.0,0.0,0.0,59,0,0,0,0,0,60,7146,M33-B01-F18_22,1,18,22,-1,28524780,2,0,"96.9,8.8,","0.9833,0.7167,","0,1,",10.902826,19.508385,0.14812975,4.720739,19.481394,0.05234097,20.761887,19.410734,0.36016145,3.124356
13,201.16205,254.97142,49.47727,201.16205,1854.9714,23.56453518259861,30.70314723396164,1,0.98333335,0.9661017,0.033898305,0.0,0.033898305,57,0,0,0,2,0,60,7830,M33-B02-F02_17,2,2,17,-1,28524831,2,0,"201.2,22.6,","0.9833,0.9333,","0,1,",15.648331,17.925293,0.050608266,13.744537,18.142805,0.0857727,12.6848545,17.062899,0.08403791,12.945444
15,611.2133,299.76617,52.04386,2486.2134,299.76617,23.577712900686066,30.52706255699551,2,0.98333335,0.9830508,0.016949153,0.0,0.016949153,116,0,0,0,2,0,120,26228,M33-B03-F07-TE_04,3,7,4,1,28526102,3,0,"113.8,56.8,20.4,","0.9833,0.9833,0.9000,","0,0,1,",11.958765,16.356699,0.01643354,42.310066,16.84331,0.02626825,41.34225,16.140528,0.0340229,31.922668
16,438.82727,246.76823,71.65176,2313.8271,1446.7682,23.605610102605468,30.691192015762407,1,0.98333335,0.8305085,0.16949153,0.06779661,0.10169491,49,0,4,0,6,0,60,27000,M33-B02-F01-TE_16,2,1,16,1,28526170,1,0,246.8,0.9833,0,20.366236,18.07968,0.10024265,6.952229,17.810106,0.0048297485,225.50786,16.615978,0.06446845,16.861229


In [5]:
ind475DS3=np.where(m33phot_test['DETSIG475']<3.0)
m33phot_test.remove_rows(ind475DS3)
m33phot_test

ID,X,Y,PIXRAD,FX,FY,RA,DEC,NUMOBJ,VIEWFRAC,CFRAC,OFRAC,GFRAC,EFRAC,CCNT,CPCNT,GCNT,GPCNT,ECNT,EPCNT,CLASSIFICATION_COUNT,PRIMARY_APCAN,SUBIMG,BRICK,FIELD,IMG,TOOTH,SUBJECT_ID,NUMOBJALL,NOSOLFLAG,EDGES,VIEWFRACS,EXCLUDEFLAGS,PIXHLR475,MAG336,SIG336,DETSIG336,MAG475,SIG475,DETSIG475,MAG814,SIG814,DETSIG814
int32,float32,float32,float32,float32,float32,float64,float64,int32,float32,float32,float32,float32,float32,int32,int32,int32,int32,int32,int32,int16,int32,bytes17,int16,int16,int16,int16,int32,int32,int16,bytes36,bytes49,bytes14,float32,float32,float32,float32,float32,float32,float32,float32,float32,float32
3,511.167,419.4067,30.805826,2386.167,2019.4067,23.60606421401875,30.699199021851136,1,1.0,1.0,0.0,0.0,0.0,60,0,0,0,0,0,60,27021,M33-B02-F01-TE_20,2,1,20,1,28526173,4,0,"80.6,4.5,6.1,20.3,","1.0000,0.2833,0.0833,0.8361,","0,1,1,1,",8.728445,20.486721,0.11886143,5.8726273,20.217335,0.1563142,6.99401,19.335989,0.27782437,3.9929368
4,274.12375,430.10718,38.054737,1524.1238,430.10718,23.43433303773043,30.51408232617703,1,1.0,0.98333335,0.016666668,0.0,0.016666668,59,0,0,0,1,0,60,21254,M33-B03-F16_03,3,16,3,-1,28525713,2,0,"69.9,28.9,","1.0000,0.9000,","0,1,",13.384977,17.207281,0.07225124,9.632259,19.698397,0.08457241,12.864533,18.16659,0.15653984,6.9838634
5,224.73451,423.41208,37.18926,849.7345,1623.4121,23.583845918691235,30.659212182808222,1,0.9836066,0.9166667,0.083333336,0.0,0.083333336,55,0,0,0,5,0,61,9815,M33-B02-F07_14,2,7,14,-1,28524968,2,0,"76.6,23.6,","0.9836,0.8548,","0,1,",9.4648075,18.9727,0.11130678,6.2656794,18.70199,0.023569176,46.085743,17.94467,0.06835622,15.904741
7,248.0658,140.99915,39.348427,873.0658,140.99915,23.390373394289195,30.6674512161078,2,0.96694213,0.974359,0.025641026,0.0,0.025641026,114,0,0,0,3,0,121,11623,M33-B02-F12_02,2,12,2,-1,28525096,2,0,"83.6,141.0,","0.9833,0.9508,","0,0,",12.779946,19.286543,0.14845498,4.710332,18.826597,0.08265972,13.160799,17.740986,0.16589268,6.5956807
8,203.50774,440.90045,63.505905,1453.5077,2440.9004,23.479685106627514,30.799434542708635,1,0.98333335,0.9830508,0.016949153,0.0,0.016949153,58,0,0,0,1,0,60,4403,M33-B01-F11_23,1,11,23,-1,28524585,2,1,"41.7,59.1,","0.9833,0.9833,","0,1,",16.818165,16.588139,0.054756697,12.701803,16.79031,0.03153818,34.436665,16.108368,0.0520216,20.886917
11,119.026726,403.0801,33.970577,744.02673,2403.08,23.445514019227883,30.7663251695561,1,0.98333335,1.0,0.0,0.0,0.0,59,0,0,0,0,0,60,7146,M33-B01-F18_22,1,18,22,-1,28524780,2,0,"96.9,8.8,","0.9833,0.7167,","0,1,",10.902826,19.508385,0.14812975,4.720739,19.481394,0.05234097,20.761887,19.410734,0.36016145,3.124356
13,201.16205,254.97142,49.47727,201.16205,1854.9714,23.56453518259861,30.70314723396164,1,0.98333335,0.9661017,0.033898305,0.0,0.033898305,57,0,0,0,2,0,60,7830,M33-B02-F02_17,2,2,17,-1,28524831,2,0,"201.2,22.6,","0.9833,0.9333,","0,1,",15.648331,17.925293,0.050608266,13.744537,18.142805,0.0857727,12.6848545,17.062899,0.08403791,12.945444
15,611.2133,299.76617,52.04386,2486.2134,299.76617,23.577712900686066,30.52706255699551,2,0.98333335,0.9830508,0.016949153,0.0,0.016949153,116,0,0,0,2,0,120,26228,M33-B03-F07-TE_04,3,7,4,1,28526102,3,0,"113.8,56.8,20.4,","0.9833,0.9833,0.9000,","0,0,1,",11.958765,16.356699,0.01643354,42.310066,16.84331,0.02626825,41.34225,16.140528,0.0340229,31.922668
16,438.82727,246.76823,71.65176,2313.8271,1446.7682,23.605610102605468,30.691192015762407,1,0.98333335,0.8305085,0.16949153,0.06779661,0.10169491,49,0,4,0,6,0,60,27000,M33-B02-F01-TE_16,2,1,16,1,28526170,1,0,246.8,0.9833,0,20.366236,18.07968,0.10024265,6.952229,17.810106,0.0048297485,225.50786,16.615978,0.06446845,16.861229


In [8]:
ind814DS3=np.where(m33phot_test['DETSIG814']<3.0)
m33phot_test.remove_rows(ind814DS3)
m33phot_test

ID,X,Y,PIXRAD,FX,FY,RA,DEC,NUMOBJ,VIEWFRAC,CFRAC,OFRAC,GFRAC,EFRAC,CCNT,CPCNT,GCNT,GPCNT,ECNT,EPCNT,CLASSIFICATION_COUNT,PRIMARY_APCAN,SUBIMG,BRICK,FIELD,IMG,TOOTH,SUBJECT_ID,NUMOBJALL,NOSOLFLAG,EDGES,VIEWFRACS,EXCLUDEFLAGS,PIXHLR475,MAG336,SIG336,DETSIG336,MAG475,SIG475,DETSIG475,MAG814,SIG814,DETSIG814
int32,float32,float32,float32,float32,float32,float64,float64,int32,float32,float32,float32,float32,float32,int32,int32,int32,int32,int32,int32,int16,int32,bytes17,int16,int16,int16,int16,int32,int32,int16,bytes36,bytes49,bytes14,float32,float32,float32,float32,float32,float32,float32,float32,float32,float32
3,511.167,419.4067,30.805826,2386.167,2019.4067,23.60606421401875,30.699199021851136,1,1.0,1.0,0.0,0.0,0.0,60,0,0,0,0,0,60,27021,M33-B02-F01-TE_20,2,1,20,1,28526173,4,0,"80.6,4.5,6.1,20.3,","1.0000,0.2833,0.0833,0.8361,","0,1,1,1,",8.728445,20.486721,0.11886143,5.8726273,20.217335,0.1563142,6.99401,19.335989,0.27782437,3.9929368
4,274.12375,430.10718,38.054737,1524.1238,430.10718,23.43433303773043,30.51408232617703,1,1.0,0.98333335,0.016666668,0.0,0.016666668,59,0,0,0,1,0,60,21254,M33-B03-F16_03,3,16,3,-1,28525713,2,0,"69.9,28.9,","1.0000,0.9000,","0,1,",13.384977,17.207281,0.07225124,9.632259,19.698397,0.08457241,12.864533,18.16659,0.15653984,6.9838634
5,224.73451,423.41208,37.18926,849.7345,1623.4121,23.583845918691235,30.659212182808222,1,0.9836066,0.9166667,0.083333336,0.0,0.083333336,55,0,0,0,5,0,61,9815,M33-B02-F07_14,2,7,14,-1,28524968,2,0,"76.6,23.6,","0.9836,0.8548,","0,1,",9.4648075,18.9727,0.11130678,6.2656794,18.70199,0.023569176,46.085743,17.94467,0.06835622,15.904741
7,248.0658,140.99915,39.348427,873.0658,140.99915,23.390373394289195,30.6674512161078,2,0.96694213,0.974359,0.025641026,0.0,0.025641026,114,0,0,0,3,0,121,11623,M33-B02-F12_02,2,12,2,-1,28525096,2,0,"83.6,141.0,","0.9833,0.9508,","0,0,",12.779946,19.286543,0.14845498,4.710332,18.826597,0.08265972,13.160799,17.740986,0.16589268,6.5956807
8,203.50774,440.90045,63.505905,1453.5077,2440.9004,23.479685106627514,30.799434542708635,1,0.98333335,0.9830508,0.016949153,0.0,0.016949153,58,0,0,0,1,0,60,4403,M33-B01-F11_23,1,11,23,-1,28524585,2,1,"41.7,59.1,","0.9833,0.9833,","0,1,",16.818165,16.588139,0.054756697,12.701803,16.79031,0.03153818,34.436665,16.108368,0.0520216,20.886917
11,119.026726,403.0801,33.970577,744.02673,2403.08,23.445514019227883,30.7663251695561,1,0.98333335,1.0,0.0,0.0,0.0,59,0,0,0,0,0,60,7146,M33-B01-F18_22,1,18,22,-1,28524780,2,0,"96.9,8.8,","0.9833,0.7167,","0,1,",10.902826,19.508385,0.14812975,4.720739,19.481394,0.05234097,20.761887,19.410734,0.36016145,3.124356
13,201.16205,254.97142,49.47727,201.16205,1854.9714,23.56453518259861,30.70314723396164,1,0.98333335,0.9661017,0.033898305,0.0,0.033898305,57,0,0,0,2,0,60,7830,M33-B02-F02_17,2,2,17,-1,28524831,2,0,"201.2,22.6,","0.9833,0.9333,","0,1,",15.648331,17.925293,0.050608266,13.744537,18.142805,0.0857727,12.6848545,17.062899,0.08403791,12.945444
15,611.2133,299.76617,52.04386,2486.2134,299.76617,23.577712900686066,30.52706255699551,2,0.98333335,0.9830508,0.016949153,0.0,0.016949153,116,0,0,0,2,0,120,26228,M33-B03-F07-TE_04,3,7,4,1,28526102,3,0,"113.8,56.8,20.4,","0.9833,0.9833,0.9000,","0,0,1,",11.958765,16.356699,0.01643354,42.310066,16.84331,0.02626825,41.34225,16.140528,0.0340229,31.922668
16,438.82727,246.76823,71.65176,2313.8271,1446.7682,23.605610102605468,30.691192015762407,1,0.98333335,0.8305085,0.16949153,0.06779661,0.10169491,49,0,4,0,6,0,60,27000,M33-B02-F01-TE_16,2,1,16,1,28526170,1,0,246.8,0.9833,0,20.366236,18.07968,0.10024265,6.952229,17.810106,0.0048297485,225.50786,16.615978,0.06446845,16.861229


In [9]:
m33goodphot_3filts_no275_110_160=m33phot_test
m33goodphot_3filts_no275_110_160.write('/Volumes/My_Passport_for_Mac/M33_M31_LGCS_Files/M33-Triangulum/Photometry_Tables/m33goodphot_3filts_no275_110_160.fits',
                 format='fits')
